In [182]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pylast
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import json
import random
from sklearn.neighbors import KNeighborsClassifier

In [145]:
playlist_features = []
valid_data_indices = []
for i in range(0, 100):
    feature_sing = pd.read_csv("./playlists/df_playlist_features_" + str(i) + ".csv")
    feature_sing = feature_sing.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    feature_sing = feature_sing.dropna()
#     print(feature_sing.tail())
    feature_sing = feature_sing.to_numpy()
    num_songs = feature_sing.shape[0]
    if (num_songs >= 30):
        valid_data_indices.append(i)
        playlist_features.append(feature_sing[:30, ].reshape((30, 13, 1)))
playlist_features = np.array(playlist_features)
print(len(valid_data_indices))
playlist_features.shape

66


(66, 30, 13, 1)

In [146]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.summary()


In [168]:
# read in labels
playlist_tags = []
with open('labels_100_playlists.json') as json_file:
    raw_label = json.load(json_file)
    for i in range(len(raw_label)):
        # filter labels with valid features
        if i in valid_data_indices:
            label = raw_label[i]
            label = np.array(list(label.values()))
            print(label)
#             label = label.reshape((-1, 1))
            playlist_tags.append(label)

    playlist_tags = np.array(playlist_tags)
    print(playlist_tags.shape)

[1.96923077e-01 5.46730769e-01 1.07884615e-01 3.80769231e-02
 4.94230769e-02 1.73846154e-01 4.09615385e-02 8.05769231e-02
 2.36730769e-01 8.15384615e-02 3.44230769e-02 3.84615385e-04
 5.96153846e-03 1.51923077e-02 3.84615385e-04 9.80769231e-03
 3.65384615e-02 1.84615385e-02 7.69230769e-04 5.86538462e-02
 2.03846154e-02 6.34615385e-03 2.69230769e-03 9.61538462e-04
 1.53846154e-03 9.80769231e-03 1.51923077e-02 8.07692308e-03
 5.26923077e-02 0.00000000e+00 3.90384615e-02 0.00000000e+00
 5.38461538e-03 9.61538462e-04 4.23076923e-03 1.76923077e-02
 1.23076923e-02 0.00000000e+00 2.50000000e-03 5.76923077e-04
 5.96153846e-03 2.19230769e-02 1.15384615e-03 1.95576923e-01
 0.00000000e+00 5.11538462e-02 3.84615385e-03 3.84615385e-03
 5.01923077e-02 5.80769231e-02 2.30769231e-02 6.92307692e-03
 2.88461538e-03 2.30769231e-03 0.00000000e+00 3.84615385e-04
 4.07692308e-02 3.24807692e-01 1.53846154e-03 1.07692308e-02
 4.61538462e-03 2.30769231e-03 2.88461538e-03 1.90384615e-02
 3.46153846e-02 3.769230

In [169]:
# train-test split get random indices
random.seed( 209 )
test_indices = random.sample(range(0, 66), 7)
print(test_indices)

# split features
train_features = []
test_features = []
for i in range(len(playlist_features)):
    if i in test_indices:
        test_features.append(playlist_features[i])
    else:
        train_features.append(playlist_features[i])

train_features = np.array(train_features)
test_features = np.array(test_features)

# split labels
train_labels = []
test_labels = []
for i in range(len(playlist_tags)):
    if i in test_indices:
        test_labels.append(playlist_tags[i])
    else:
        train_labels.append(playlist_tags[i])

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

[2, 21, 36, 5, 41, 43, 11]


In [230]:
# build CNN structure
input_shape = train_features[0].shape
num_category = train_labels[0].shape[0]

# model building
model = models.Sequential()
#convolutional layer with rectified linear unit activation
model.add(layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters with each size 3x3
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features by adding a pooling layer
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(layers.Dropout(0.25))
#flatten layer
model.add(layers.Flatten())
#fully connected to get all relevant data
model.add(layers.Dense(128, activation='relu'))
#one more dropout for convergence
model.add(layers.Dropout(0.5))
#output multiple sigmoid to generate output probabilities
model.add(layers.Dense(num_category, activation='sigmoid'))

model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 28, 11, 32)        320       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 26, 9, 64)         18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 13, 4, 64)         0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 13, 4, 64)         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 3328)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               426112    
_________________________________________________________________
dropout_40 (Dropout)         (None, 128)             

In [232]:
# Use adaDelta as the adaptive learning rate
model.compile(loss=tf.keras.losses.mean_absolute_error,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [237]:
# Train Model
# too few data to have a batch_size
#batch_size = 128
num_epoch = 30
#model training
model_log = model.fit(train_features, train_labels,
          epochs=num_epoch,
          verbose=1,
          validation_data=(test_features, test_labels))

Train on 59 samples, validate on 7 samples
Epoch 1/30
59/59 [==============================] - 0s 1ms/sample - loss: 0.5172 - accuracy: 0.1864 - val_loss: 0.5266 - val_accuracy: 0.4286
Epoch 2/30
59/59 [==============================] - 0s 1ms/sample - loss: 0.5181 - accuracy: 0.1356 - val_loss: 0.5266 - val_accuracy: 0.4286
Epoch 3/30
59/59 [==============================] - 0s 852us/sample - loss: 0.5214 - accuracy: 0.1186 - val_loss: 0.5266 - val_accuracy: 0.4286
Epoch 4/30
59/59 [==============================] - 0s 912us/sample - loss: 0.5263 - accuracy: 0.1525 - val_loss: 0.5266 - val_accuracy: 0.4286
Epoch 5/30
59/59 [==============================] - 0s 837us/sample - loss: 0.5196 - accuracy: 0.1356 - val_loss: 0.5266 - val_accuracy: 0.4286
Epoch 6/30
59/59 [==============================] - 0s 833us/sample - loss: 0.5143 - accuracy: 0.2034 - val_loss: 0.5266 - val_accuracy: 0.4286
Epoch 7/30
59/59 [==============================] - 0s 867us/sample - loss: 0.5154 - accuracy: 0.

In [238]:
score = model.evaluate(train_features, train_labels, verbose=1)
print(score)
print('Train loss:', score[0]) 
print('Train accuracy:', score[1]) 

59/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [239]:
model.predict(train_features)

array([[0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       ...,
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [240]:
score = model.evaluate(test_features, test_labels, verbose=1)
print(score)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1]) 

7/1 [==================================================================================================================================================================================================================] - 0s 547us/sample - loss: 0.5265 - accuracy: 0.4286
[0.5265222191810608, 0.42857143]
Test loss: 0.5265222191810608
Test accuracy: 0.42857143


In [241]:
# KNN Model
# ks = [1, 2, 3, 4, 5, 7, 9, 10, 15, 25]
# for k in ks:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(train_features, train_labels)
#     cur_score = np.mean(cross_val_score(knn, train_features, train_labels, cv=5))

#     if cur_score > max_score:
#         max_score = cur_score
#         max_k = k

# print()